In [ ]:
!pip install langchain
!pip install neo4j
!pip install bs4
!pip install llama-index
!pip uninstall -y trulens_eval
#!pip install trulens-eval==0.25.1
!pip install llmlingua
!pip install regex

In [ ]:
!pip uninstall -y trulens_eval
!pip install trulens-eval

In [ ]:
import os
from bs4 import BeautifulSoup
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
import warnings
from langchain_community.document_loaders import DirectoryLoader
from llama_index.core import SimpleDirectoryReader, StorageContext
warnings.filterwarnings("ignore")
from langchain_community.embeddings import HuggingFaceEmbeddings
from sentence_transformers import CrossEncoder
import numpy as np
from trulens_eval import Tru
from trulens_eval.tru_custom_app import instrument
from llmlingua import PromptCompressor
import re
from IPython.core.display import display, HTML

In [ ]:
def word_wrap(string, n_chars=72):
    # Wrap a string at the next space after n_chars
    if len(string) < n_chars:
        return string
    else:
        return string[:n_chars].rsplit(' ', 1)[0] + '\n' + word_wrap(string[len(string[:n_chars].rsplit(' ', 1)[0])+1:], n_chars)

In [ ]:
NEO4J_URI = 'bolt://' + os.getenv('NEO4J_HOST') + ':7687'
NEO4J_USERNAME = os.getenv('NEO4J_USER')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = 'neo4j' #os.getenv('NEO4J_DB')
print(NEO4J_URI)
print(NEO4J_DATABASE)

In [ ]:
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

In [ ]:
cypher = """
  MATCH (n) 
  RETURN count(n)
  """
result = kg.query(cypher)
result

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    separators=["\n\n", "\n", ". ", " ", ""],
)

In [ ]:
file_metadata = lambda x: {"filename": x}
Acts_documents = SimpleDirectoryReader("./XML",file_metadata=file_metadata).load_data()

## Working with the 1st section

#### bcl - B.C Laws #####
bcl:num gives the number of sections, subsections and something more

bcl:section gives all the sections and subsections

bcl:marginalnote gices the section heading

if the section is a definition then each definition has a definition tag

In [ ]:
section_1 = num_sections[2]
section_heading = section_1.find_all('bcl:marginalnote')
section_definitions = section_1.find_all('bcl:definition')
section_subsection = section_1.find_all('bcl:subsection')
print(section_1)
print(len(section_subsection))

In [ ]:
merge_chunk_node_query = """
MERGE(mergedChunk:UpdatedChunk {chunkId: $chunkParam.chunkId})
    ON CREATE SET 
        mergedChunk.chunkSeqId = $chunkParam.chunkSeqId, 
        mergedChunk.text = $chunkParam.text,
        mergedChunk.ActId = $chunkParam.ActId,
        mergedChunk.sectionId = $chunkParam.sectionId,
        mergedChunk.sectionName = $chunkParam.sectionName
RETURN mergedChunk
"""

In [ ]:
match_chunk_node_query =  """
        MATCH (chunk:UpdatedChunk) WHERE
        chunk.chunkId = $chunkParam.chunkId
        AND chunk.chunkSeqId = $chunkParam.chunkSeqId
        AND chunk.ActId = $chunkParam.ActId
        AND chunk.sectionId = $chunkParam.sectionId
        AND chunk.sectionName = $chunkParam.sectionName
        AND chunk.text = $chunkParam.text
        RETURN chunk
        """

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
create_embeddings = """
        MATCH (chunk:UpdatedChunk) WHERE
        chunk.chunkId = $chunkParam.chunkId
        AND chunk.chunkSeqId = $chunkParam.chunkSeqId
        AND chunk.ActId = $chunkParam.ActId
        AND chunk.sectionId = $chunkParam.sectionId
        AND chunk.sectionName = $chunkParam.sectionName
        AND chunk.text = $chunkParam.text
        AND chunk.textEmbedding is NULL
        CALL db.create.setNodeVectorProperty(chunk, "textEmbedding", $vector)
        RETURN chunk
    """


In [ ]:
connect_chunks = """
      MATCH (chunk:UpdatedChunk), (f:UpdatedChunk)
      WHERE
        chunk.chunkId = $chunkParam.chunkId
        AND chunk.chunkSeqId = $chunkParam.chunkSeqId
        AND chunk.ActId = $chunkParam.ActId
        AND chunk.sectionId = $chunkParam.sectionId
        AND chunk.sectionName = $chunkParam.sectionName
        AND f.ActId = $chunkParam.ActId
        AND f.sectionId = $chunkParam.connectnedsectionId
        AND f.chunkSeqId = 0
      MERGE (chunk)-[newRelationship:REFERENCE]->(f)
      RETURN count(newRelationship)
    """

In [ ]:
def create_chunk_relationship(act_info):
    cypher = """
  MATCH (from_same_section:UpdatedChunk)
  WHERE from_same_section.ActId = $ActParam['ActId']
  AND from_same_section.sectionName = $ActParam['sectionName']
  AND from_same_section.sectionId = $ActParam['sectionId']
  WITH from_same_section
    ORDER BY from_same_section.chunkSeqId ASC
  WITH collect(from_same_section) as section_chunk_list
    CALL apoc.nodes.link(
        section_chunk_list, 
        "NEXT", 
        {avoidDuplicates: true}
    )  // NEW!!!
  RETURN size(section_chunk_list)
"""
    kg.query(cypher, params={'ActParam': act_info})

In [ ]:
## This function will search for any reference that has the word section or subsection followed by a number
def extract_references(str, index):
    references = re.findall(r"section (\d+)", str)
      

In [ ]:
def find_links(subsection):
    if not subsection:
        return
    if (subsection.find_all("bcl:link")):
        xml_link = subsection.find_all("bcl:link")[0]['xlink:href']
        display(HTML(f'<a href="{xml_link}">{subsection.find_all("bcl:link")[0].get_text()}</a>'))
    extract_references(subsection.get_text().replace("\n\n", "").replace("\r", ""), index)
    return subsection.get_text().replace("\n", " ").replace("\r", " ")

In [ ]:
#get subsection
def subsection(section_definitions, index):
    string = ""
    for subsection_index, subsection in enumerate(section_definitions):
        string += "\n" + find_links(subsection)
    return string

In [ ]:
def create_metadata(token_split_texts,title, section_heading, section_id):
    chunks_with_metadata = [] # use this to accumlate chunk records
    chunk_seq_id = 0
    for chunk in token_split_texts: # only take the first 20 chunks
        #form_id = file[file.rindex('/') + 1:file.rindex('.')] # extract form id from file name
        # finally, construct a record with metadata and the chunk text
        chunks_with_metadata.append({
            'text': chunk, 
            # metadata from looping...
            'chunkSeqId': chunk_seq_id,
            'chunkId': f'{title}-chunk-{section_heading}-{chunk_seq_id:04d}',
            'ActId': f'{title}',
            'sectionId': f'{section_id}',
            'sectionName':f'{section_heading}',
            # constructed metadata...
            # metadata from file...
        })
        chunk_seq_id += 1
    return chunks_with_metadata

In [ ]:
def create_chunks(item_text, title, section_heading, section_id):
    item_text_chunks = text_splitter.split_text(item_text) # split the text into chunks
    token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=20, tokens_per_chunk=256)
    token_split_texts = []
    for text in item_text_chunks:
        token_split_texts += token_splitter.split_text(text)
    meta_data = create_metadata(token_split_texts,title, section_heading, section_id)    
    return meta_data

In [ ]:
def search_create_reference(match):
    #print(match[0]['chunk'])
    text = match[0]['chunk']['text']
    references = re.findall(r"(?i)(?:section)\s+(\d+|\(\d+\))(?:\s*\([a-z]\))?", text)
    if (references):
        print(match[0]['chunk'])
        print(references)
        # create the edges
        print("Match found - creating references")
        chunk_seq_id = match[0]['chunk']['chunkSeqId']
        section_heading = match[0]['chunk']['sectionName']
        section_id = match[0]['chunk']['sectionId']
        chunk = {
            'text': text, 
            # metadata from looping...
            'chunkSeqId': match[0]['chunk']['chunkSeqId'],
            'chunkId': match[0]['chunk']['chunkId'],
            'ActId': match[0]['chunk']['ActId'],
            'sectionId': match[0]['chunk']['sectionId'],
            'sectionName': match[0]['chunk']['sectionName'],
            'connectnedsectionId': references[0]
        }
        result = kg.query(connect_chunks,
                params={
                    'chunkParam':chunk
                }
                )
        print(result)

In [ ]:
def create_chunk_embeddings(tokens):
    for chunk in tokens:
        print(f"Creating `:Chunk` node for chunk ID {chunk['chunkSeqId']}")
        kg.query(merge_chunk_node_query, 
            params={
                'chunkParam': chunk
            })
        vector = embeddings.embed_query(chunk['text'])
        result = kg.query(create_embeddings, params={'chunkParam':chunk, 'vector':vector})
        if result:
            print("Embedding created")
        else:
            print(result)
            print("Embedding not created")
    create_chunk_relationship(tokens[0])       

In [ ]:
def create_chunk_neo4j(tokens, search=False):
    for chunk in tokens:
        if search:
            vector = embeddings.embed_query(chunk['text'])
            result = kg.query(create_embeddings, params={'chunkParam':chunk, 'vector':vector})
            if result:
                print("Embedding creating")
            else:
                print(result)
                return
        else:
            print(f"Creating `:Chunk` node for chunk ID {chunk['chunkSeqId']}")
            kg.query(merge_chunk_node_query, 
                    params={
                        'chunkParam': chunk
                    })
    if (not search):
        create_chunk_relationship(tokens[0])

In [ ]:
def extract_data(file, index,  search=False):
    #get the ACT's title
    title = file.find('act:title')
    if title:
        title = title.get_text()
        print(title)
    else:
        return
    #get all the sections
    preamble = file.find_all('bcl:preamble')
    if (preamble):
        item_text = subsection(preamble, 0)
        token = create_chunks(item_text, title, 'preamble', 0)
    sections = file.find_all('bcl:section')
    #find the definition subsection
    for index, section in enumerate(sections):
        token = []
        section_heading = section.find('bcl:marginalnote')#[0].get_text()
        if (section_heading):
            section_heading = section_heading.get_text()
        else:
            section_heading = ""
        section_definitions = section.find_all('bcl:definition')
        if (len(section_definitions) < 1):
            #find the remaining subsection
            section_subsection = section.find_all('bcl:subsection')
            #find the section number
            section_number = section.find('bcl:num').get_text()
            if len(section_number):
                print("section number is:" + section_number)
            #print(section_subsection)
            if len(section_subsection):
                item_text = subsection(section_subsection, index+1)
                print(item_text)
                token = create_chunks(item_text, title, section_heading, section_number)
            else:
                item_text = find_links(section)
                token = create_chunks(item_text, title, section_heading, section_number)
        else:
            item_text = subsection(section_definitions, index+1)
            #print(item_text)
            token = create_chunks(item_text, title, section_heading, index+1)
        found = create_chunk_embeddings(token)

# This cell will create chunks based on sections of the Acts

In [ ]:
for index, Acts in enumerate(Acts_documents):
    soup = BeautifulSoup(Acts.get_text(), 'xml')
    extract_data(soup, index,  False)